# 03-Calculate Bearings

This notebook estimates the bearings (azimuth) for all locations in the vehicle trips. The bearing is the direction of movement measured in decimal degrees, with zero facing north and angles increasing in the clockwise direction. 

In [ ]:
%run calculate-bearings.py